<a href="https://colab.research.google.com/github/rickvelloso/Learning-IA/blob/main/MultiLabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import hamming_loss
from sklearn.multioutput import ClassifierChain
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [3]:
musica = pd.read_csv('Musica.csv')
musica.head(4)

,amazed-suprised,happy-pleased,relaxing-clam,quiet-still,sad-lonely,angry-aggresive,Mean_Acc1298_Mean_Mem40_Centroid,Mean_Acc1298_Mean_Mem40_Rolloff,Mean_Acc1298_Mean_Mem40_Flux,Mean_Acc1298_Mean_Mem40_MFCC_0,...,Std_Acc1298_Std_Mem40_MFCC_10,Std_Acc1298_Std_Mem40_MFCC_11,Std_Acc1298_Std_Mem40_MFCC_12,BH_LowPeakAmp,BH_LowPeakBPM,BH_HighPeakAmp,BH_HighPeakBPM,BHSUM1,BHSUM2,BHSUM3
0,0,1,1,0,0,0,0.132498,0.077848,0.229227,0.602629,...,0.197026,0.196244,0.164323,0.030017,0.253968,0.008473,0.240602,0.136735,0.058442,0.107594
1,1,0,0,0,0,1,0.384281,0.355249,0.167190,0.853089,...,0.093526,0.085649,0.025101,0.182955,0.285714,0.156764,0.270677,0.191377,0.153728,0.197951
2,0,1,0,0,0,1,0.541782,0.356491,0.152246,0.791142,...,0.198082,0.108067,0.140574,0.099303,0.142857,0.000000,0.593985,0.105114,0.025555,0.122965
3,0,0,1,0,0,0,0.174288,0.243935,0.254326,0.438987,...,0.235793,0.220195,0.235834,0.024988,0.222222,0.117169,0.210526,0.057288,0.134575,0.091509


In [4]:
musica.shape

(592, 77)

In [6]:
classe = musica.iloc[:,0:6].values
previsores = musica.iloc[:,7:78].values

In [7]:
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

In [8]:
previsores

array([[-0.8981437 ,  0.59645542, -0.03960736, ..., -0.49591226,
        -0.69187266, -0.854754  ],
       [ 0.66080303,  0.00318568,  1.37433704, ..., -0.20667729,
        -0.21862043, -0.38517106],
       [ 0.66778286, -0.13972618,  1.02462206, ..., -0.66329082,
        -0.85521088, -0.77487132],
       ...,
       [-1.10473431, -1.13171275, -1.15410898, ..., -0.67620641,
        -0.29937817, -0.74471328],
       [ 0.36627331,  1.10524422,  0.26285006, ...,  0.39361683,
        -0.47519758,  0.45232124],
       [-0.46329804, -0.55724418, -0.19985176, ..., -0.96218602,
        -0.3278917 , -0.78358664]])

In [10]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores, classe, test_size=0.3, random_state=5)

In [12]:
svm = SVC()
classifier_chain = ClassifierChain(base_estimator=svm, random_state=5)
classifier_chain.fit(X_treinamento, y_treinamento)

ClassifierChain(base_estimator=SVC(), random_state=5)

In [24]:
y_pred = classifier_chain.predict(X_teste)
loss = hamming_loss(y_teste, y_pred)
print(f"Hamming Loss: {loss}")
#18% das classificações ficaram erradas utilizando este modelo.

Hamming Loss: 0.18258426966292135


Fazendo melhorias com GridSearch para melhorar os parâmetros e buscar melhores resultados nas métricas.

In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
svm = SVC()
classifier_chain_grid = ClassifierChain(base_estimator=svm, random_state=5)

In [16]:
parametros = {
    'base_estimator__C': [0.1, 1, 10],
    'base_estimator__kernel': ['linear', 'rbf']
}

Com o classificador e parâmetros definidos, partimos para configurar o gridsearch.


In [18]:
grid_search = GridSearchCV(estimator=classifier_chain_grid,
                           param_grid=parametros,
                           cv=3, n_jobs=-1, verbose=2)

In [21]:
print("--- Iniciando a busca pelos melhores parâmetros... ---")
grid_search.fit(X_treinamento, y_treinamento)

--- Iniciando a busca pelos melhores parâmetros... ---
Fitting 3 folds for each of 6 candidates, totalling 18 fits


GridSearchCV(cv=3,
             estimator=ClassifierChain(base_estimator=SVC(), random_state=5),
             n_jobs=-1,
             param_grid={'base_estimator__C': [0.1, 1, 10],
                         'base_estimator__kernel': ['linear', 'rbf']},
             verbose=2)

In [22]:
print("Melhores parâmetros encontrados:")
print(grid_search.best_params_)

Melhores parâmetros encontrados:
{'base_estimator__C': 1, 'base_estimator__kernel': 'rbf'}


In [23]:
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_teste)
loss_best = hamming_loss(y_teste, y_pred_best)

print(f"\nHamming Loss (Original): {loss:.4f}")
print(f"Hamming Loss (Otimizado): {loss_best:.4f}")


Hamming Loss (Original): 0.1826
Hamming Loss (Otimizado): 0.1826


Observação que apesar de tentar uma otimização dos parâmetros para obter uma melhor performance o resultado foi o mesmo.
Possivel motivo: os parâmetros já eram os melhores ou o modelo atingiu o máximo de desempenho (menos provavel).
Agora irei testar outras otimizações para ver se é possivel melhorar, mesmo que pouco, essa performance.

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [27]:
rf = RandomForestClassifier(n_estimators=100, random_state=5, n_jobs=-1)
classifier_chain_rf = ClassifierChain(base_estimator=rf, random_state=5)

In [28]:
classifier_chain_rf.fit(X_treinamento, y_treinamento)

ClassifierChain(base_estimator=RandomForestClassifier(n_jobs=-1,
                                                      random_state=5),
                random_state=5)

In [29]:
y_pred_rf = classifier_chain_rf.predict(X_teste)

loss_rf = hamming_loss(y_teste, y_pred_rf)

print(f"Hamming Loss (SVC Otimizado): {loss_best:.4f}")
print(f"Hamming Loss (RandomForest):  {loss_rf:.4f}")

Hamming Loss (SVC Otimizado): 0.1826
Hamming Loss (RandomForest):  0.1685


Desempenho com o RandomForest foi superior em pouco mais de 1%.
Sinal que não estavamos extraindo tudo que os dados eram capazes de nos oferecer, mesmo com a otimização, portanto o problema era que ali era o máximo que o modelo poderia nos oferecer.
Por que o RandomForest teve um desempenho superior?
Por ser um conjunto de diversas arvores de decisão é ótimo para encontrar interações complexas e não-lineares, como estavamos tratando de 70 atributos ele acabou relacionando-os melhor (com base na métrica de hamming loss).
Agora vamos tentar otimizar ainda mais o RnadomForest para ver se conseguimos um resultado ainda superior utilizando o GridSearch.

In [30]:
rf_grid = RandomForestClassifier(random_state=5, n_jobs=-1)
classifier_chain_rf_grid = ClassifierChain(base_estimator=rf_grid, random_state=5)
parametros_rf = {
    'base_estimator__n_estimators': [100, 200],  # 100 (padrão) vs 200
    'base_estimator__max_depth': [None, 10]      # 'None' (árvores completas) vs 10 (árvores curtas)
}

In [31]:
grid_search_rf = GridSearchCV(estimator=classifier_chain_rf_grid,
                              param_grid=parametros_rf,
                              cv=3,
                              n_jobs=-1,
                              verbose=2)

In [32]:
print("--- Iniciando a busca pelos melhores parâmetros do RandomForest... ---")
grid_search_rf.fit(X_treinamento, y_treinamento)

--- Iniciando a busca pelos melhores parâmetros do RandomForest... ---
Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3,
             estimator=ClassifierChain(base_estimator=RandomForestClassifier(n_jobs=-1,
                                                                             random_state=5),
                                       random_state=5),
             n_jobs=-1,
             param_grid={'base_estimator__max_depth': [None, 10],
                         'base_estimator__n_estimators': [100, 200]},
             verbose=2)

In [33]:
print("Melhores parâmetros encontrados:")
print(grid_search_rf.best_params_)

Melhores parâmetros encontrados:
{'base_estimator__max_depth': 10, 'base_estimator__n_estimators': 100}


In [34]:
best_model_rf = grid_search_rf.best_estimator_
y_pred_best_rf = best_model_rf.predict(X_teste)
loss_best_rf = hamming_loss(y_teste, y_pred_best_rf)

print(f"\nHamming Loss (RandomForest Padrão): {loss_rf:.4f}")
print(f"Hamming Loss (RandomForest Otimizado): {loss_best_rf:.4f}")


Hamming Loss (RandomForest Padrão): 0.1685
Hamming Loss (RandomForest Otimizado): 0.1742


Apesar de ter tido uma leve queda de acuracia ~0.005 houve uma boa otimização no quesito eficiência. no modelo antigo as arvores cresceram o maximo possivel (max_depth=none e n_estimators=100), o que pode aumentar os riscos de overfitting e se tornam mais lentas para treinos.
Com o gridsearch utilizando um max_depth10 tivemos um resultado muito próximo no quesito acuracia, porém com um modelo mais rapido e mais leve. Além disso, pela diferença de acuracia ser tão baixa, é possivel tratar até como uma diferença de ruido estatistico.
Agora vamos mudar a estatégia para continuar explorando essa base de dados e extrair o máximo possivel dela.

In [36]:
!pip install scikit-multilearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 4.7 MB/s eta 0:00:00


In [37]:
from skmultilearn.problem_transform import LabelPowerset

Agora vamos a um teste com LabelPowerset no lugar do chain e utilizando o randomforest.

In [38]:
rf_lp = RandomForestClassifier(n_estimators=100, random_state=5, n_jobs=-1)
modelo_lp_rf = LabelPowerset(
    classifier=rf_lp,
    require_dense=[False, True]
)

In [39]:
modelo_lp_rf.fit(X_treinamento, y_treinamento)

LabelPowerset(classifier=RandomForestClassifier(n_jobs=-1, random_state=5),
              require_dense=[False, True])

In [40]:
y_pred_lp_rf = modelo_lp_rf.predict(X_teste)

loss_lp_rf = hamming_loss(y_teste, y_pred_lp_rf)

print(f"Hamming Loss (Chain + RF Padrão): {loss_rf:.4f}") # O 0.1685 anterior
print(f"Hamming Loss (Powerset + RF):   {loss_lp_rf:.4f}")

Hamming Loss (Chain + RF Padrão): 0.1685
Hamming Loss (Powerset + RF):   0.1639


Agora com a nova estratégia (labelpowerset no lugar do classifierchain) obtivemos um resultado levemente superior. Iremos agora fazer o mesmo ajuste de eficiência utilizando o GridSearch e comparando para ver qual é mais válido.

In [41]:
rf_grid_lp = RandomForestClassifier(random_state=5, n_jobs=-1)

In [42]:
modelo_lp_rf_grid = LabelPowerset(
    classifier=rf_grid_lp,
    require_dense=[False, True]
)

In [43]:
parametros_lp_rf = {
    'classifier__n_estimators': [100, 200],  # Acessamos o parâmetro do RF com 'classifier__'
    'classifier__max_depth': [None, 10]
}

In [45]:
grid_search_lp_rf = GridSearchCV(estimator=modelo_lp_rf_grid,
                                 param_grid=parametros_lp_rf,
                                 cv=3,
                                 n_jobs=-1,
                                 verbose=2)

In [46]:
grid_search_lp_rf.fit(X_treinamento, y_treinamento)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3,
             estimator=LabelPowerset(classifier=RandomForestClassifier(n_jobs=-1,
                                                                       random_state=5),
                                     require_dense=[False, True]),
             n_jobs=-1,
             param_grid={'classifier__max_depth': [None, 10],
                         'classifier__n_estimators': [100, 200]},
             verbose=2)

In [47]:
print("\nMelhores parâmetros encontrados:")
print(grid_search_lp_rf.best_params_)


Melhores parâmetros encontrados:
{'classifier__max_depth': 10, 'classifier__n_estimators': 200}


In [48]:
best_model_lp_rf = grid_search_lp_rf.best_estimator_
y_pred_best_lp_rf = best_model_lp_rf.predict(X_teste)
loss_best_lp_rf = hamming_loss(y_teste, y_pred_best_lp_rf)

print(f"\nHamming Loss (Powerset + RF Padrão): {loss_lp_rf:.4f}")
print(f"Hamming Loss (Powerset + RF Otimizado): {loss_best_lp_rf:.4f}")


Hamming Loss (Powerset + RF Padrão): 0.1639
Hamming Loss (Powerset + RF Otimizado): 0.1676


Novamente o gridsearch trouxe uma melhora na a eficiência e com uma confiância muito proxima (menor que 0.4% de 0.1639 vs 0.1676).
Além disso, ao que tudo indica estamos no limite da otimização para essa arquitetura, troquei o motor e estratégia, além de ter feito ajuste nos parâmetros e ainda assim parece que estamos nessa barreira de pouco mais de 15% (no começo estavamos em 18.25 agora estamos em 16.40 numa versão menos eficiente e 16.76 numa versão mais eficiente.)
Próximo passo será fazer uma seleção de atributos para tentar reduzir o ruido e buscar uma melhora real na confiança.

In [56]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif

In [65]:
classe_fs = musica.iloc[:, 0:6].values
previsores_fs = musica.iloc[:, 7:78].values
X_treino_fs, X_teste_fs, y_treino_fs, y_teste_fs = train_test_split(previsores_fs, classe_fs, test_size=0.3, random_state=5)

In [66]:
y_treino_df = pd.DataFrame(y_treino_fs).astype(str)
y_treino_strings = y_treino_df.apply(lambda x: '-'.join(x), axis=1)

In [69]:
le = LabelEncoder()
y_treino_1d_para_selecao = le.fit_transform(y_treino_strings)

In [70]:
scaler = StandardScaler()
X_treino_scaled = scaler.fit_transform(X_treino_fs)
X_teste_scaled = scaler.transform(X_teste_fs)

In [83]:
selector = SelectKBest(f_classif, k=40)

In [84]:
selector.fit(X_treino_scaled, y_treino_1d_para_selecao)

SelectKBest(k=40)

In [85]:
X_treino_selected = selector.transform(X_treino_scaled)
X_teste_selected = selector.transform(X_teste_scaled)

print(f"Formato original de X: {X_treino_scaled.shape}")
print(f"Formato novo de X: {X_treino_selected.shape}")

Formato original de X: (414, 70)
Formato novo de X: (414, 40)


In [86]:
rf_final = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=5, n_jobs=-1)

model_final = LabelPowerset(
    classifier=rf_final,
    require_dense=[False, True]
)

In [87]:
model_final.fit(X_treino_selected, y_treino_fs)

LabelPowerset(classifier=RandomForestClassifier(max_depth=10, n_jobs=-1,
                                                random_state=5),
              require_dense=[False, True])

In [88]:
y_pred_final = model_final.predict(X_teste_selected)
loss_final = hamming_loss(y_teste_fs, y_pred_final)

# (loss_lp_rf é a variável do resultado anterior 0.1639)
print(f"\nHamming Loss (Powerset + RF Padrão): {loss_lp_rf:.4f}")
print(f"Hamming Loss (Seleção(k=40) + Powerset + RF): {loss_final:.4f}")


Hamming Loss (Powerset + RF Padrão): 0.1639
Hamming Loss (Seleção(k=40) + Powerset + RF): 0.1770


Removendo 31 atributos (de 71 para 40) tivemos uma queda de confiança de 0.1639 para 0.1770, o que demonstra que não eram atributos de ruido.
Dessa forma da para fazer um resumo das comparações que tivemos durante todo o notebook:

| # | Estratégia | "Motor" (Classificador) | Seleção de Atributos | Hamming Loss (Erro) | Conclusão |
|---|---|---|---|---|---|
| 1 | ClassifierChain | SVC (Padrão) | Não (71 atributos) | 0.1826 | Nosso Baseline |
| 2 | ClassifierChain | SVC (Otimizado) | Não (71 atributos) | 0.1826 | Platô do SVC |
| 3 | ClassifierChain | RandomForest (Padrão) | Não (71 atributos) | 0.1685 | Melhor! O motor RF é superior. |
| 4 | ClassifierChain | RandomForest (Otimizado) | Não (71 atributos) | 0.1742 | Sem ganho. Otimização não ajudou. Porém com melhor eficiência em questão de tempo de processamento. |
| 5 | LabelPowerset | RandomForest (Padrão) | Não (71 atributos) | 0.1639 | CAMPEÃO! A estratégia LP é superior. |
| 6 | LabelPowerset | RandomForest (Otimizado) | Não (71 atributos) | 0.1676 | Sem ganho. Atingimos o limite do modelo. Porém com melhor eficiência em questão de tempo de processamento. |
| 7 | LabelPowerset | RandomForest (Otimizado) | Sim (40 atributos) | 0.1770 | Piorou! A seleção de atributos é prejudicial. |

Celulas repetindo o que ja foi feito com classifier chain e lpset para verificar se a minha ideia de ganho de eficiência realmente foi real com a otimização de parametros. (podia ter feito isso nas celulas la em cima? sim, mas acabei fazendo aqui e repetindo código. é isso rs)

In [89]:
# --- CÉLULA NOVA: Comparação de Tempo (ClassifierChain + RF) ---
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import hamming_loss

# --- Modelo 1: Padrão (max_depth=None) ---
print("\n--- Teste de Tempo: Chain + RF Padrão (max_depth=None) ---")

rf_padrao = RandomForestClassifier(n_estimators=100, random_state=5, n_jobs=-1, max_depth=None)
chain_rf_padrao = ClassifierChain(base_estimator=rf_padrao, random_state=5)

# Medir Tempo de Treino
inicio_treino = time.time()
chain_rf_padrao.fit(X_treinamento, y_treinamento)
fim_treino = time.time()
tempo_treino_padrao = fim_treino - inicio_treino

# Medir Tempo de Previsão
inicio_pred = time.time()
y_pred_rf_padrao = chain_rf_padrao.predict(X_teste)
fim_pred = time.time()
tempo_pred_padrao = fim_pred - inicio_pred

loss_rf_padrao = hamming_loss(y_teste, y_pred_rf_padrao)
print(f"Hamming Loss: {loss_rf_padrao:.4f}")
print(f"Tempo de Treino: {tempo_treino_padrao:.4f} segundos")
print(f"Tempo de Previsão: {tempo_pred_padrao:.4f} segundos")


# --- Modelo 2: Otimizado (max_depth=10) ---
print("\n--- Teste de Tempo: Chain + RF Otimizado (max_depth=10) ---")

# (O modelo que o GridSearch encontrou, que deu 0.1742)
rf_otimizado = RandomForestClassifier(n_estimators=100, random_state=5, n_jobs=-1, max_depth=10)
chain_rf_otimizado = ClassifierChain(base_estimator=rf_otimizado, random_state=5)

# Medir Tempo de Treino
inicio_treino = time.time()
chain_rf_otimizado.fit(X_treinamento, y_treinamento)
fim_treino = time.time()
tempo_treino_otimizado = fim_treino - inicio_treino

# Medir Tempo de Previsão
inicio_pred = time.time()
y_pred_rf_otimizado = chain_rf_otimizado.predict(X_teste)
fim_pred = time.time()
tempo_pred_otimizado = fim_pred - inicio_pred

loss_rf_otimizado = hamming_loss(y_teste, y_pred_rf_otimizado)
print(f"Hamming Loss: {loss_rf_otimizado:.4f}")
print(f"Tempo de Treino: {tempo_treino_otimizado:.4f} segundos")
print(f"Tempo de Previsão: {tempo_pred_otimizado:.4f} segundos")


--- Teste de Tempo: Chain + RF Padrão (max_depth=None) ---
Hamming Loss: 0.1685
Tempo de Treino: 3.5983 segundos
Tempo de Previsão: 0.8751 segundos

--- Teste de Tempo: Chain + RF Otimizado (max_depth=10) ---
Hamming Loss: 0.1742
Tempo de Treino: 3.5073 segundos
Tempo de Previsão: 0.4123 segundos


In [90]:
# --- CÉLULA NOVA: Comparação de Tempo (LabelPowerset + RF) ---
import time
from skmultilearn.problem_transform import LabelPowerset

# --- Modelo 3: Padrão (max_depth=None) ---
print("\n--- Teste de Tempo: Powerset + RF Padrão (max_depth=None) ---")

rf_lp_padrao = RandomForestClassifier(n_estimators=100, random_state=5, n_jobs=-1, max_depth=None)
model_lp_padrao = LabelPowerset(classifier=rf_lp_padrao, require_dense=[False, True])

# Medir Tempo de Treino
inicio_treino = time.time()
model_lp_padrao.fit(X_treinamento, y_treinamento)
fim_treino = time.time()
tempo_treino_lp_padrao = fim_treino - inicio_treino

# Medir Tempo de Previsão
inicio_pred = time.time()
y_pred_lp_padrao = model_lp_padrao.predict(X_teste)
fim_pred = time.time()
tempo_pred_lp_padrao = fim_pred - inicio_pred

loss_lp_padrao = hamming_loss(y_teste, y_pred_lp_padrao)
print(f"Hamming Loss: {loss_lp_padrao:.4f}")
print(f"Tempo de Treino: {tempo_treino_lp_padrao:.4f} segundos")
print(f"Tempo de Previsão: {tempo_pred_lp_padrao:.4f} segundos")


# --- Modelo 4: Otimizado (max_depth=10, n_estimators=200) ---
print("\n--- Teste de Tempo: Powerset + RF Otimizado (GridSearch) ---")

# (O modelo que o GridSearch encontrou, que deu 0.1676)
rf_lp_otimizado = RandomForestClassifier(n_estimators=200, random_state=5, n_jobs=-1, max_depth=10)
model_lp_otimizado = LabelPowerset(classifier=rf_lp_otimizado, require_dense=[False, True])

# Medir Tempo de Treino
inicio_treino = time.time()
model_lp_otimizado.fit(X_treinamento, y_treinamento)
fim_treino = time.time()
tempo_treino_lp_otimizado = fim_treino - inicio_treino

# Medir Tempo de Previsão
inicio_pred = time.time()
y_pred_lp_otimizado = model_lp_otimizado.predict(X_teste)
fim_pred = time.time()
tempo_pred_lp_otimizado = fim_pred - inicio_pred

loss_lp_otimizado = hamming_loss(y_teste, y_pred_lp_otimizado)
print(f"Hamming Loss: {loss_lp_otimizado:.4f}")
print(f"Tempo de Treino: {tempo_treino_lp_otimizado:.4f} segundos")
print(f"Tempo de Previsão: {tempo_pred_lp_otimizado:.4f} segundos")


--- Teste de Tempo: Powerset + RF Padrão (max_depth=None) ---
Hamming Loss: 0.1639
Tempo de Treino: 0.9830 segundos
Tempo de Previsão: 0.0523 segundos

--- Teste de Tempo: Powerset + RF Otimizado (GridSearch) ---
Hamming Loss: 0.1676
Tempo de Treino: 1.8000 segundos
Tempo de Previsão: 0.0768 segundos


| # | Estratégia | "Motor" (Classificador) | Hamming Loss (Erro) | Tempo de Treino (s) | Tempo de Previsão (s) | Conclusão |
|---|---|---|---|---|---|---|
| 1 | ClassifierChain | SVC (Otimizado) | 0.1826 | - | - | Baseline (Lento) |
| 2 | ClassifierChain | RF (max_depth=10) | 0.1742 | 3.51s | 0.41s | Rápido na previsão, mas erro alto |
| 3 | ClassifierChain | RF (max_depth=None) | 0.1685 | 3.60s | 0.88s | Erro bom, mas o mais lento |
| 4 | LabelPowerset | RF (max_depth=None) | 0.1639 | 0.98s | 0.05s | VENCEDOR EM TUDO |
| 5 | LabelPowerset | RF (max_depth=10) | 0.1676 | 1.80s | 0.08s | Quase tão bom, mas mais lento |

okok, com relação ao classfierchain ele realmente ficou mais rapido com o ajuste de profundidae de arvores, porém o LP PIOROU com o ajuste de profundidade (na verdade a piora foi pelo aumento dos n_estimators para 200) e além de ter sido mais lento foi pior em confiança, portanto o LPSET com o gridsearchCV foi definitivamente o pior.
Ao menos, todas essas etapas serviram para refinar o máximo os modelos e extair o máximo possível dos dados.

LabelPowerset (sem otimização com GridSearchCV) foi o campeão em tempo de treino, tempo de previsão e confiança!